In [1]:
from datasets import load_dataset
import seaborn as sns
import matplotlib.pyplot as plt
import torch

from transformers import BertForQuestionAnswering, AdamW

In [2]:
dataset = load_dataset('squad')
train_dataset, validation_dataset = dataset['train'], dataset['validation']

Reusing dataset squad (/Users/christianbauer/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 145.46it/s]


In [3]:


pretrained_model_name: str = 'bert-large-uncased-whole-word-masking-finetuned-squad'

full_bert = BertForQuestionAnswering.from_pretrained(pretrained_model_name)
# full_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
def get_last_saved_epoch(path: str = 'models/') -> int:
    from os import walk
    files = []
    for (_, _, filenames) in walk(path):
        filenames = [file for file in filenames if file.endswith('.pt')]
        files.extend(filenames)

    epoch_list = [int(file.split('.')[0].split('_').pop()) for file in files]

    return max(epoch_list) if len(epoch_list) > 0 else -1


def init_model(epoch: int = 0):
    model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

    # model.to(device)
    
    if epoch != -1:
        model_dict = torch.load(f'models/bert_model_{epoch}.pt', map_location=torch.device('cpu'))
        model.load_state_dict(model_dict)

    return model

In [6]:
saved_epoch = get_last_saved_epoch()
my_model = init_model(saved_epoch)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [7]:
full_bert.eval()
my_model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_